# Analysis of Low-field Density Scan for PSI Paper
This is the most up to date (as of 2020-07-02) analysis of the density scan data, with temperature minimisation implemented, to make figures to put into the PSI paper

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import xarray as xr 
import scipy.stats as stat
import collections
import sys
import os
import glob
import re
import importlib
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.core.ivdata as iv
import flopter.core.lputils as lp
import flopter.magnum.database as ut
import flopter.magnum.utils as mgut
import flopter.core.fitters as fts
import flopter.core.constants as c

In [2]:
import importlib
importlib.reload(mgut)

<module 'flopter.magnum.utils' from '/home/jleland/coding/projects/flopter/flopter/magnum/utils.py'>

In [3]:
# Create analysed dataset metadata 

path_to_datasets = '/home/jleland/data/external/magnum/'
# path_to_datasets = '/home/jleland/data/externy/magnum/'
# path_to_analysed_datasets = 'analysed_2'
# path_to_analysed_datasets = 'analysed_3'
# path_to_analysed_datasets = 'phobos_test'
# path_to_analysed_datasets = 'test'
# path_to_analysed_datasets = 'analysed_3_downsampled'
# path_to_analysed_datasets = 'analysed_4'
# path_to_analysed_datasets = 'analysed_4_downsampled'
path_to_analysed_datasets = 'analysed_5_downsampled'
os.chdir(path_to_datasets)

In [4]:
magnum_probes = lp.MagnumProbes()

## Load adc file metadata

In [5]:
os.chdir('/home/jleland/data/external/magnum/')
# os.chdir('/home/jleland/data/externy/magnum/')
meta_data_ds = xr.open_dataset('all_meta_data.nc')
# print(meta_data_ds)

### Indices for different Density Scans

In [6]:
# 0.8T Scans
super_title = 'H scan @ 0.8T, tilt=10'
indices_n_e_08T_H = [124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,142,143,144,145,146,147]

super_title = 'H scan @ 0.8T, tilt=8'
indices_n_e_08T_H_8deg = [276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,301,302,303,304,305,306,307,308,309,310,311]

# Other Scans
super_title = 'H scan @ 0.8T, tilt=10'
indices_n_e_12T_H = [199,200,201,202,203,204,205,206,207,208,209,210]

super_title = 'H scan @ 0.8T, tilt=10'
indices_n_e_15T_H = [389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415]

super_title = 'H axial scan @ 0.8T, tilt=10'
axial_scan_indices = [132,148,149,150,151,152,153,154,155,156,157,158,159,160,161,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189]

In [7]:
droppable_vars = {
    'adc_4_probe',
    'adc_5_probe',
    'adc_calibration_index',
    'adc_end_time',
    'adc_filename',
    'adc_folder',
    'adc_index',
    'adc_time',
    'shot_end_time',
    'sweep_time',
    'ts_number',
    'ts_time',
    'ts_timestamp'
}

In [9]:
mgut.get_dataset_metadata(path_to_analysed_datasets).loc[indices_n_e_08T_H]

,shot_number,shot_timestamp,shot_time,filename,time_len,sweep_len
adc_index,,,,,,
124,119,6698264154516119552,2019-06-03 12:25:54,analysed_5_downsampled/a119_124_66982641545161...,50,998
125,120,6698265184455914496,2019-06-03 12:29:54,analysed_5_downsampled/a120_125_66982651844559...,50,999
126,121,6698266156670986240,2019-06-03 12:33:40,analysed_5_downsampled/a121_126_66982661566709...,50,999
127,122,6698267042814349312,2019-06-03 12:37:06,analysed_5_downsampled/a122_127_66982670428143...,50,998
128,123,6698267851003597824,2019-06-03 12:40:15,analysed_5_downsampled/a123_128_66982678510035...,50,998
129,124,6698268912468647936,2019-06-03 12:44:22,analysed_5_downsampled/a124_129_66982689124686...,50,999
130,125,6698270567416166400,2019-06-03 12:50:47,analysed_5_downsampled/a125_130_66982705674161...,50,998
131,126,6698273241866715136,2019-06-03 13:01:10,analysed_5_downsampled/a126_131_66982732418667...,50,998
132,127,6698274668406413312,2019-06-03 13:06:42,analysed_5_downsampled/a127_132_66982746684064...,50,999


In [13]:
importlib.reload(mgut)

<module 'flopter.magnum.utils' from '/home/jleland/coding/projects/flopter/flopter/magnum/utils.py'>

In [14]:
density_scan_ds = mgut.get_dataset_from_indices(indices_n_e_08T_H_8deg + indices_n_e_08T_H, anglescan_fl=False, 
                                                average_direction_fl=False,
                                                path_to_analysed_datasets=path_to_analysed_datasets,
#                                                 preprocess='average_coarsen',
#                                                 check_sweep_trim_fl=True,
                                                ).drop(droppable_vars)

In [15]:
density_scan_ds['shot_target_pos']

<xarray.DataArray 'shot_target_pos' (shot_number: 56)>
array([-0.199951, -0.199951, -0.199951, -0.199951, -0.199951, -0.199951,
       -0.199951, -0.199951, -0.199951, -0.199951, -0.199951, -0.199951,
       -0.199951, -0.199951, -0.199951, -0.199951, -0.199951, -0.199951,
       -0.199951, -0.199951, -0.199951, -0.199951, -0.199951, -0.199951,
       -0.199951, -0.199951, -0.199951, -0.199951, -0.199951, -0.199951,
       -0.199951, -0.199951, -0.199951, -0.199951, -0.199951, -0.199951,
       -0.199951, -0.199951, -0.199951, -0.199951, -0.199951, -0.199951,
       -0.199951, -0.199951, -0.199951, -0.199951, -0.199951, -0.199951,
       -0.199951, -0.199951, -0.199951, -0.199951, -0.199951, -0.199951,
       -0.199951, -0.199951])
Coordinates:
  * shot_number  (shot_number) int32 283 284 285 286 287 ... 138 139 140 141 142
    tilt         (shot_number) float64 8.0 8.0 8.0 8.0 ... 10.0 10.0 10.0 10.0

In [16]:
def plot_densscan_ivs(densscan_ds):
    probes = np.array(densscan_ds.probe.values, ndmin=1)
    
    if len(probes) == 1:
        fig, ax = plt.subplots()
        densscan_ds.set_coords('voltage')['current'].plot.line(x='voltage', hue='shot_number', ax=ax)
    else:
        fig, ax = plt.subplots(len(probes))
        for i, probe in enumerate(probes):
            ds = densscan_ds.sel(probe=probe)
            ds.set_coords('voltage')['current'].plot.line(x='voltage', hue='shot_number', ax=ax[i])
            
    return fig, ax

def plot_densscan_timeseries(densscan_ds):
    probes = np.array(densscan_ds.probe.values, ndmin=1)
    
    if len(probes) == 1:
        fig, ax = plt.subplots(2, sharex=True)
        densscan_ds['current'].plot.line(x='time', hue='shot_number', ax=ax[0])
        densscan_ds['voltage'].plot.line(x='time', hue='shot_number', ax=ax[1])
        
    else:
        fig, ax = plt.subplots(2, len(probes), sharex=True)
        for i, probe in enumerate(probes):
            ds = densscan_ds.sel(probe=probe)
            ds['current'].plot.line(x='time', hue='shot_number', ax=ax[i][0])
            ds['voltage'].plot.line(x='time', hue='shot_number', ax=ax[i][1])
            
    return fig, ax

In [17]:
mgut.plot_densscan_paramspace(density_scan_ds)
mgut.plot_densscan_multi_ts(density_scan_ds)

In [18]:
plot_densscan_ivs(density_scan_ds)
# plot_densscan_timeseries(density_scan_ds)

(<Figure size 640x480 with 2 Axes>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x7f2d7e9b0390>,
       dtype=object))

In [19]:
# ds = density_scan_ds.swap_dims({'shot_number':'tilt'}).sel(tilt=8.0).swap_dims({'tilt':'shot_number'})
ds = density_scan_ds.sel(shot_number=slice(314,317))
plot_densscan_ivs(ds)
plot_densscan_timeseries(ds)

(<Figure size 640x480 with 4 Axes>,
 array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7f2d6823ba58>,
       dtype=object))

In [42]:
probe_ds = density_scan_ds.where(density_scan_ds.tilt == 8.0, drop=True).sel(probe='L')
# probe_ds = probe_ds.where(probe_ds['current'].min('time') > -0.1, drop=True)
plot_densscan_ivs(probe_ds)
# shot_numbers = probe_ds.shot_number.values

(<Figure size 640x480 with 1 Axes>,
 <matplotlib.axes._subplots.AxesSubplot at 0x7f9d237bd828>)

In [20]:
fit_df = mgut.fit_magnum_ds(density_scan_ds, scan_param='shot_number', threshold=None, plot_fl=False,  sat_region=-40,
#                             scan_selection={'shot_number':shot_numbers}, 
                            probes=('S', 'L'), mode=0, trimming_vals=(0.1, 0.25, 0.02))
# fit_df = mgut.fit_magnum_ds(density_scan_ds, scan_param='shot_number', probes=('S'), threshold=None, plot_fl=False,  sat_region=-40, mode=0, trimming_vals=(0.2, 0.06, 0.02))

In [21]:
combined_ds = mgut.combine_fit_ds(fit_df, density_scan_ds, probes=('L', 'S'), index_dim='shot_number')

In [22]:
combined_ds

<xarray.Dataset>
Dimensions:                 (probe: 2, shot_number: 56, time: 50, ts_radial_pos: 46)
Coordinates:
  * probe                   (probe) object 'L' 'S'
  * shot_number             (shot_number) int32 283 284 285 286 ... 140 141 142
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    tilt                    (shot_number) float64 8.0 8.0 8.0 ... 10.0 10.0 10.0
Dimensions without coordinates: time
Data variables:
    voltage                 (shot_number, probe, time) float64 -94.33 ... -88.71
    current                 (shot_number, probe, time) float64 -0.007033 ... -0.4715
    shot_time               (shot_number, probe, time) float64 5.372 ... 4.336
    start_time              (shot_number, probe) float64 5.37 5.37 ... 4.333
    d_current               (shot_number, probe, time) float64 8.568e-05 ... 0.0005915
    std_current             (shot_number, probe, time) float64 0.003582 ... 0.02159
    ts_density              (shot_number, ts_radial_pos) float64 ...
    ts_temperature          (shot_number, ts_radial_pos) float64 ...
    ts_d_density            (shot_number, ts_radial_pos) float64 ...
    ts_d_temperature        (shot_number, ts_radial_pos) float64 ...
    ts_temp_max             (shot_number) float64 ...
    ts_dens_max             (shot_number) float64 ...
    ts_fl                   (shot_number) bool ...
    shot_source_current     (shot_number) float64 ...
    shot_b_field            (shot_number) float64 ...
    shot_tilt               (shot_number) float64 7.988 7.988 ... 9.991 9.991
    shot_target_pos         (shot_number) float64 -0.2 -0.2 -0.2 ... -0.2 -0.2
    shot_target_rot         (shot_number) float64 ...
    shot_target_pumpspeed   (shot_number) int32 ...
    shot_source_pumpspeed   (shot_number) int32 ...
    shot_heating_pumpspeed  (shot_number) int32 ...
    shot_source_pos         (shot_number) float64 ...
    shot_deuterium_gf       (shot_number) float64 ...
    shot_hydrogen_gf        (shot_number) float64 ...
    shot_helium_gf          (shot_number) float64 ...
    shot_hydrogen_puffed    (shot_number) float64 ...
    shot_helium_puffed      (shot_number) float64 ...
    adc_timestamp           (shot_number) float64 ...
    adc_4_coax              (shot_number) float64 ...
    adc_4_shunt_resistance  (shot_number) float64 ...
    adc_5_coax              (shot_number) float64 ...
    adc_5_shunt_resistance  (shot_number) float64 ...
    adc_calibration_fl      (shot_number) float64 ...
    shot_fl                 (shot_number) float64 ...
    acquisition_length      (shot_number) float64 ...
    adc_end_timestamp       (shot_number) float64 ...
    adc_freqs               (shot_number) float64 ...
    B                       (probe, shot_number) float64 0.8 0.8 0.8 ... 0.8 0.8
    ts_temp                 (probe, shot_number) object 2.3658216933436154 ... 0.7846797657184977
    ts_dens                 (probe, shot_number) object 1.1910288184069587e+19 ... 3.54035214688245e+20
    fit_success_fl          (probe, shot_number) object True True ... True True
    temp                    (probe, shot_number) float64 1.47 2.393 ... 1.816
    d_temp                  (probe, shot_number) float64 0.3985 ... 0.1523
    isat                    (probe, shot_number) float64 0.00321 ... 0.3774
    d_isat                  (probe, shot_number) float64 0.0001086 ... 0.001683
    a                       (probe, shot_number) float64 0.06046 ... 0.01295
    d_a                     (probe, shot_number) float64 0.01154 ... 0.0007811
    v_f                     (probe, shot_number) float64 -19.52 ... -4.447
    d_v_f                   (probe, shot_number) float64 0.141 ... 0.1667
    dens                    (probe, shot_number) float64 3.252e+17 ... 4.575e+19
    d_dens                  (probe, shot_number) float64 4.964e+16 ... 3.613e+18
    chi2                    (probe, shot_number) float64 242.5 ... 1.403e+03
    reduced_chi2            (probe, shot_number) float64

In [25]:
accepted_shots = list(set(combined_ds.shot_number.values) - set(erroneous_shots))
# erroneous_ds = combined_ds
combined_ds = combined_ds.sel(shot_number=accepted_shots)
# print(erroneous_ds)
# print(combined_ds)

In [26]:
s_shots = combined_ds.sel(probe='S').sortby('ts_dens')
l_shots = combined_ds.sel(probe='L').sortby('ts_dens')

In [27]:
mgut.plot_densscan_paramspace(s_shots)

In [28]:
fig, ax = plt.subplots()

# for tilt in [8.0, 10.0]:
#     combined_ds.swap_dims({'shot_number': 'tilt'}).sel(tilt=tilt).set_coords(['ts_temp', 'ts_dens']).plot.scatter(x='ts_dens', y='ts_temp', ax=ax)
combined_ds.set_coords(['tilt', 'ts_temp', 'ts_dens']).plot.scatter(x='ts_dens', y='ts_temp', hue='tilt', ax=ax)

In [29]:
fig, ax = plt.subplots(2, sharex=True, sharey=True)

s_shots.set_coords(['ts_temp_max', 'ts_dens_max']).plot.scatter(x='ts_dens_max', y='ts_temp_max', ax=ax[0])
s_shots.set_coords(['ts_temp', 'ts_dens']).plot.scatter(x='ts_dens', y='ts_temp', ax=ax[1])

In [39]:
full_ts_ds = xr.open_dataset('full_ts_dataset.nc')

shot_number_ds = meta_data_ds.dropna('shot_number', subset=['adc_index']).swap_dims({'shot_number':'adc_index'}).sel(adc_index=indices_n_e_08T_H+indices_n_e_08T_H_8deg)
shot_numbers = shot_number_ds.shot_number

densscan_ts = full_ts_ds.sel(ts_time=np.isin(full_ts_ds.shot_number, shot_numbers))
# densscan_ts

mean_percentage_dens_ts = (densscan_ts['ts_d_density'] / densscan_ts['ts_density']).mean('ts_time').values
mean_percentage_temp_ts = (densscan_ts['ts_d_temperature'] / densscan_ts['ts_temperature']).mean('ts_time').values

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [40]:
for shot_number in densscan_ts.shot_number.values[0:5]:
    shot_ds = densscan_ts.where(densscan_ts.shot_number == shot_number, drop=True)
    md_shot_ds = meta_data_ds.sel(shot_number=shot_number)
    
    fig, ax = plt.subplots(2, sharex=True)
    fig.suptitle(f'{shot_number} - {md_shot_ds.shot_tilt.values}')
    
    ax[0].plot('ts_radial_pos', 'ts_d_temperature', data=md_shot_ds, label='averaged')
    ax[1].plot('ts_radial_pos', 'ts_d_density', data=md_shot_ds, label='averaged')
    
    ax[0].plot('ts_radial_pos', 'ts_d_temperature', data=shot_ds.std('ts_time'), label='averaged_alt')
    ax[1].plot('ts_radial_pos', 'ts_d_density', data=shot_ds.std('ts_time'), label='averaged_alt')
    
    for ts_time in shot_ds.ts_time.values:
        ds = shot_ds.sel(ts_time=ts_time)
        
        ax[0].plot('ts_radial_pos', 'ts_d_temperature', data=ds, label='raw')
        ax[1].plot('ts_radial_pos', 'ts_d_density', data=ds, label='raw')
        
        ax[0].plot(ds['ts_radial_pos'].values, ds['ts_temperature'].values * mean_percentage_temp_ts, label='estimated')
        ax[1].plot(ds['ts_radial_pos'].values, ds['ts_density'].values * mean_percentage_dens_ts, label='estimated')
        
    for axis in ax:
        axis.legend()

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [58]:
ts_ds = densscan_ts.groupby('shot_number').first().sel(shot_number=combined_ds.shot_number)

In [69]:
for ts_var in ['ts_temperature', 'ts_d_temperature', 'ts_density', 'ts_d_density']:
    combined_ds[ts_var] = ts_ds[ts_var]

In [70]:
fig, ax = plt.subplots()

markers = ['^', '*']
scan_indices = [indices_n_e_08T_H, indices_n_e_08T_H_8deg]
scan_tilts = [10.0, 8.0]
# scan_indices = [indices_n_e_15T_H, indices_n_e_15T_H]
labels = [r' - 10$^{\circ}$ tilt', r' - 8$^{\circ}$ tilt']

for i in range(2):
    ts_scan = combined_ds.swap_dims({'shot_number':'tilt'}).sel(tilt=scan_tilts[i]).max('ts_radial_pos')
    s_shots_scan = s_shots.sel(shot_number=ts_scan['shot_number']).max('ts_radial_pos')
    l_shots_scan = l_shots.sel(shot_number=ts_scan['shot_number']).max('ts_radial_pos')

    ax.errorbar(s_shots_scan['ts_density'], s_shots_scan['temp'], yerr=s_shots_scan['d_temp'], label='S'+labels[i],
                color='tab:blue', ecolor='tab:blue', linestyle='none', marker=markers[i], mfc='none')
    ax.errorbar(l_shots_scan['ts_density'], l_shots_scan['temp'], yerr=l_shots_scan['d_temp'], label='L'+labels[i], 
                color='tab:orange', ecolor='tab:orange', linestyle='none', marker=markers[i], mfc='none')
    ax.errorbar(ts_scan['ts_density'], ts_scan['ts_temperature'], yerr=ts_scan['ts_d_temperature'].values, 
                color='tab:green', ecolor='tab:green', label='Thomson', linestyle='none', marker=markers[i],
                mfc='none')
    ax.set_xscale('log')
    ax.legend()

In [52]:
# Plot comparing the density scans of hydrogen at 0.8T on the S and L probes at two slightly different tilts
# with plt.style.context('seaborn'):

fig, ax = plt.subplots(2, 2, sharex=True, constrained_layout=True)

colours = {
    'L': 'blue',
    'S': 'red',
    'B': 'green',
    'R': 'gold',
    'TS': 'black'
}
markers = ['^', '*']
scan_indices = [indices_n_e_08T_H, indices_n_e_08T_H_8deg]
scan_tilts = [10.0, 8.0]
# scan_indices = [indices_n_e_15T_H, indices_n_e_15T_H]
labels = [r' - 10$^{\circ}$ tilt', r' - 8$^{\circ}$ tilt']
leg_labels = []
leg_handles = []

for i in range(2):
    ts_scan = combined_ds.swap_dims({'shot_number':'tilt'}).sel(tilt=scan_tilts[i]).max('ts_radial_pos')
    s_shots_scan = s_shots.sel(shot_number=ts_scan['shot_number']).max('ts_radial_pos')
    l_shots_scan = l_shots.sel(shot_number=ts_scan['shot_number']).max('ts_radial_pos')
    slabel = f'S{labels[i]}'
    llabel = f'L{labels[i]}'
    
    shandle = ax[0][0].errorbar(s_shots_scan['ts_density'], s_shots_scan['temp'], 
                                yerr=s_shots_scan['d_temp'], label=slabel,
                                color=colours['S'], ecolor=colours['S'], linestyle='none', 
                                marker=markers[i], mfc='none')
    
    lhandle = ax[0][0].errorbar(l_shots_scan['ts_density'], l_shots_scan['temp'], 
                                yerr=l_shots_scan['d_temp'], label=llabel, 
                                color=colours['L'], ecolor=colours['L'], 
                                linestyle='none', marker=markers[i], mfc='none')
    tslabel='Thomson'
    tshandle = ax[0][0].errorbar(ts_scan['ts_density'], ts_scan['ts_temperature'], 
                                 yerr=ts_scan['ts_d_temperature'].values, 
                                 color=colours['TS'], ecolor=colours['TS'], 
                                 label=tslabel, linestyle='none', marker=markers[i],
                                 mfc='none')
    ax[0][0].set_xscale('log')
#     ax[0][0].legend()
    
    if i == 0:
        leg_labels = [tslabel]
        leg_handles = [tshandle]
    leg_labels.extend([slabel, llabel])
    leg_handles.extend([shandle, lhandle])
    

    ax[0][1].errorbar(s_shots_scan['ts_density'], s_shots_scan['dens'], 
                      yerr=s_shots_scan['d_dens'], label=slabel,
                      color=colours['S'], ecolor=colours['S'], 
                      linestyle='none', marker=markers[i], mfc='none')
    ax[0][1].errorbar(l_shots_scan['ts_density'], l_shots_scan['dens'], 
                      yerr=l_shots_scan['d_dens'], label=llabel, 
                      color=colours['L'], ecolor=colours['L'], 
                      linestyle='none', marker=markers[i], mfc='none')
    ax[0][1].errorbar(ts_scan['ts_density'], ts_scan['ts_density'], 
                      yerr=ts_scan['ts_d_density'].values, 
                      color=colours['TS'], ecolor=colours['TS'], 
                      label=tslabel, linestyle='none', 
                      marker=markers[i], mfc='none')
    ax[0][1].set_yscale('log')
    ax[0][1].set_xscale('log')
#     ax[0][1].legend()

    ax[1][0].plot(s_shots_scan['ts_density'], s_shots_scan['temp'] / s_shots_scan['ts_temperature'], label='S'+labels[i], 
                  color=colours['S'], linestyle='none', marker=markers[i], mfc='none')
    ax[1][0].plot(l_shots_scan['ts_density'], l_shots_scan['temp'] / l_shots_scan['ts_temperature'], label='L'+labels[i], 
                  color=colours['L'], linestyle='none', marker=markers[i], mfc='none')
    ax[1][0].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
    ax[1][0].set_xscale('log')
#     ax[1][0].legend()

    ax[1][1].plot(s_shots_scan['ts_density'], s_shots_scan['dens'] / s_shots_scan['ts_density'], 
                  label=slabel, color=colours['S'], linestyle='none', marker=markers[i], mfc='none')
    ax[1][1].plot(l_shots_scan['ts_density'], l_shots_scan['dens'] / l_shots_scan['ts_density'], 
                  label=llabel, color=colours['L'], linestyle='none', marker=markers[i], mfc='none')
    ax[1][1].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
    ax[1][1].set_xscale('log')
#     ax[1][1].legend()

fig.suptitle('Plots of temperature and density measurements during the density scan at 0.8T in hydrogen')
ax[0][0].set_title('Temperature')
ax[0][0].set_ylabel(r'$T_e$ (eV)')
ax[0][1].set_title('Density')
ax[0][1].set_ylabel(r'$n_e$ (m$^{-3}$)')

ax[1][0].set_ylabel(r'$T_{e,probe}$ / $T_{e,TS}$')
ax[1][0].set_xlabel(r'$n_{e,TS}$ (m$^{-3}$)')
ax[1][1].set_xlabel(r'$n_{e,TS}$ (m$^{-3}$)')
ax[1][1].set_ylabel(r'$n_{e,probe}$ / $n_{e,TS}$')
fig.legend(leg_handles, leg_labels, loc='center left', bbox_to_anchor=(0.8,0.5))

In [108]:
importlib.reload(mgut)

<module 'flopter.magnum.utils' from '/home/jleland/coding/projects/flopter/flopter/magnum/utils.py'>

In [33]:
# combined_ds
ts_probe_temps, ts_probe_denss, ts_probe_d_temps, ts_probe_d_denss = mgut.interpolate_ts_position(combined_ds, aggregate_dims=None, offset=1.75)

len(ts_probe_temps['L'][:,0])
len(ts_scan['ts_density'])

33

In [82]:
fig, ax = plt.subplots(1, 2, sharey=True, constrained_layout=True, figsize=[8,5])

colours = {
    'L': 'blue',
    'S': 'red',
    'B': 'green',
    'R': 'gold',
    'TS': 'black'
}
markers = ['^', 's']
scan_indices = [indices_n_e_08T_H, indices_n_e_08T_H_8deg]
scan_tilts = [10.0, 8.0]
# scan_indices = [indices_n_e_15T_H, indices_n_e_15T_H]
labels = [r' - 10$^{\circ}$ tilt', r' - 8$^{\circ}$ tilt']
leg_labels = []
leg_handles = []

for i in range(2):
    ts_scan = combined_ds.swap_dims({'shot_number':'tilt'}).sel(tilt=scan_tilts[i])
    s_shots_scan = s_shots.sel(shot_number=ts_scan['shot_number']).max('ts_radial_pos')
    l_shots_scan = l_shots.sel(shot_number=ts_scan['shot_number']).max('ts_radial_pos')
    
    ts_probe_temps, ts_probe_denss, ts_probe_d_temps, ts_probe_d_denss = mgut.interpolate_ts_position(ts_scan, aggregate_dims=None)
    ts_scan = ts_scan.max('ts_radial_pos')
    
    s_q_par = (s_shots_scan['isat'].values / magnum_probes['S'].get_collection_area(np.radians(scan_tilts[i]))) * 7 * s_shots_scan['temp'].values
    l_q_par = (l_shots_scan['isat'].values / magnum_probes['L'].get_collection_area(np.radians(scan_tilts[i]))) * 7 * l_shots_scan['temp'].values
    ts_q_par = ((lp.sound_speed(s_shots_scan['ts_temp'].astype(np.float64).values) 
                 * s_shots_scan['ts_dens'].values * 1.6e-19) * 7 * s_shots_scan['ts_temp'].values)
    
    slabel = f'S{labels[i]}'
    llabel = f'L{labels[i]}'
    
    ax[0].plot(ts_probe_denss['S'][:, 0], s_shots_scan['temp'] / ts_probe_temps['S'][:, 0], 
               label=slabel, color=colours['S'], linestyle='none', marker=markers[i], mfc='none')
    ax[0].plot(ts_probe_denss['L'][:, 0], l_shots_scan['temp'] / ts_probe_temps['L'][:, 0], 
               label=llabel, color=colours['L'], linestyle='none', marker=markers[i], mfc='none')
    
    ax[1].plot(s_q_par, s_shots_scan['temp'] / ts_probe_temps['S'][:, 0], 
               label=slabel, color=colours['S'], linestyle='none', marker=markers[i], mfc='none')
    ax[1].plot(l_q_par, l_shots_scan['temp'] / ts_probe_temps['L'][:, 0], 
               label=llabel, color=colours['L'], linestyle='none', marker=markers[i], mfc='none')

#     ax.plot(s_shots_scan['ts_density'], s_shots_scan['temp'] / ts_scan['ts_temperature'].values, 
#             label=slabel, color=colours['S'], linestyle='none', marker=markers[i], mfc='none')
#     ax.plot(l_shots_scan['ts_density'], l_shots_scan['temp'] / ts_scan['ts_temperature'].values, 
#             label=llabel, color=colours['L'], linestyle='none', marker=markers[i], mfc='none')

#     ax.errorbar(ts_scan['ts_density'], ts_scan['ts_temperature'], 
#                 yerr=ts_scan['ts_d_temperature'].values, 
#                 color=colours['TS'], ecolor=colours['TS'], 
#                 label=tslabel, linestyle='none', marker=markers[i],
#                 mfc='none')
#     ax.errorbar(ts_scan['ts_density'], ts_probe_temps['S'][:,0], 
#                 yerr=ts_probe_d_temps['S'][:,0], 
#                 color=colours['TS'], ecolor=colours['TS'], 
#                 label=tslabel, linestyle='none', marker=markers[i],
#                 mfc='none')
#     ax.errorbar(ts_scan['ts_density'], ts_probe_temps['L'][:,0], 
#                 yerr=ts_probe_d_temps['L'][:,0], 
#                 color=colours['TS'], ecolor=colours['TS'], 
#                 label=tslabel, linestyle='none', marker=markers[i],
#                 mfc='gold')
    ax[0].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
    ax[0].set_xscale('log')
#     ax.set_yscale('log')
    ax[0].legend()
    
    ax[1].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
    ax[1].set_xscale('log')
#     ax.set_yscale('log')
    ax[1].legend()
    
ax[0].set_title('(a)')
ax[1].set_title('(b)')

ax[0].set_ylabel(r'$T_{e,probe}$ [eV]')
# ax[1].set_ylabel(r'$T_{e,probe}$ [eV]')
# ax[0].set_ylabel(r'$Q_{\parallel,TS}$ [Wm$^{-2}$]')
# ax[1].set_ylabel(r'$T_{e,probe}$ / $T_{e,TS}$')
# ax[1][1].set_ylabel(r'$n_{e,probe}$ / $n_{e,TS}$')

ax[0].set_xlabel(r'$n_{e,TS}$ (m$^{-3}$)')
ax[1].set_xlabel(r'$Q_{\parallel,TS}$ [Wm$^{-2}$]')

Text(0.5, 0, '$Q_{\\parallel,TS}$ [Wm$^{-2}$]')

In [72]:
fig, ax = plt.subplots(constrained_layout=True, figsize=[5,5])

leg_labels = []
leg_handles = []
trunc_ds = combined_ds.where(combined_ds.ts_density < 2e20, drop=True)
# trunc_ds = combined_ds.sel(shot_number=trunc_ds_shots)

mfcs = [
    [colours['S'], colours['L']],
    ['none', 'none'],
]

for i in range(2):
    ts_scan = trunc_ds.swap_dims({'shot_number':'tilt'}).sel(tilt=scan_tilts[i])
    s_shots_scan = s_shots.sel(shot_number=ts_scan['shot_number']).max('ts_radial_pos')
    l_shots_scan = l_shots.sel(shot_number=ts_scan['shot_number']).max('ts_radial_pos')
    
    ts_probe_temps, ts_probe_denss, ts_probe_d_temps, ts_probe_d_denss = mgut.interpolate_ts_position(ts_scan, aggregate_dims=None)
    ts_scan = ts_scan.max('ts_radial_pos')
    
    sshots_temp = s_shots_scan['temp'] / ts_probe_temps['S'][:, 0]
    d_sshots_temp = error_prop(sshots_temp, s_shots_scan['temp'], ts_probe_temps['S'][:,0],
                               s_shots_scan['d_temp'], ts_probe_d_temps['S'][:,0])
    lshots_temp = l_shots_scan['temp'] / ts_probe_temps['L'][:, 0]
    d_lshots_temp = error_prop(lshots_temp, l_shots_scan['temp'], ts_probe_temps['L'][:,0],
                               l_shots_scan['d_temp'], ts_probe_d_temps['L'][:,0])
    
    slabel = f'S{labels[i]}'
    llabel = f'L{labels[i]}'
    
    ax.errorbar(ts_probe_denss['S'][:, 0], sshots_temp, yerr=d_sshots_temp, 
                label=slabel, color=colours['S'], linestyle='none', marker='^', mfc=mfcs[i][0])
    ax.errorbar(ts_probe_denss['L'][:, 0], lshots_temp, yerr=d_lshots_temp,
                label=llabel, color=colours['L'], linestyle='none', marker='s', mfc=mfcs[i][1])
    
    ax.axhline(y=1.0, **c.AX_LINE_DEFAULTS)
    ax.set_xscale('log')
    ax.legend()

ax.set_ylabel(r'$T_{e,probe}$ / $T_{e,TS}$')
ax.set_xlabel(r'$n_{e,TS}$ (m$^{-3}$)')

l 6
s -4
b -14
r -24
l 6
s -4
b -14
r -24


Text(0.5, 0, '$n_{e,TS}$ (m$^{-3}$)')

In [104]:
# max: 200A, 8.4
# min: 100A, 7.6
density_scan_ds.min('shot_number')[['shot_source_current', 'shot_hydrogen_gf']]

<xarray.Dataset>
Dimensions:              ()
Data variables:
    shot_source_current  float64 99.9
    shot_hydrogen_gf     float64 7.58

In [54]:
def error_prop(f, a, b, da, db):
    return np.abs(f) * np.sqrt((da / a)**2 + (db / b)**2)

In [92]:
fig, ax = plt.subplots(2, sharex=True, constrained_layout=True, figsize=[5,5])

leg_labels = []
leg_handles = []
trunc_ds = combined_ds.where(combined_ds.ts_density < 2e20, drop=True)

for i in range(2):
    ts_scan = trunc_ds.swap_dims({'shot_number':'tilt'}).sel(tilt=scan_tilts[i])
    s_shots_scan = s_shots.sel(shot_number=ts_scan['shot_number']).max('ts_radial_pos')
    l_shots_scan = l_shots.sel(shot_number=ts_scan['shot_number']).max('ts_radial_pos')
    
    ts_probe_temps, ts_probe_denss, ts_probe_d_temps, ts_probe_d_denss = mgut.interpolate_ts_position(ts_scan, aggregate_dims=None)
    ts_scan = ts_scan.max('ts_radial_pos')
    
    slabel = f'S{labels[i]}'
    llabel = f'L{labels[i]}'
    
    ax[0].errorbar(ts_probe_denss['S'][:, 0], s_shots_scan['temp'] / ts_probe_temps['S'][:, 0],
                   label=slabel, color=colours['S'], linestyle='none', marker=markers[i], mfc='none')
    ax[0].plot(ts_probe_denss['L'][:, 0], l_shots_scan['temp'] / ts_probe_temps['L'][:, 0], 
               label=llabel, color=colours['L'], linestyle='none', marker=markers[i], mfc='none')
    
    ax[0].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
    ax[0].set_xscale('log')
    ax[0].legend()
    
    ax[1].plot(ts_probe_denss['S'][:, 0], s_shots_scan['dens'] / ts_probe_denss['S'][:, 0], 
               label=slabel, color=colours['S'], linestyle='none', marker=markers[i], mfc='none')
    ax[1].plot(ts_probe_denss['L'][:, 0], l_shots_scan['dens'] / ts_probe_denss['L'][:, 0], 
               label=llabel, color=colours['L'], linestyle='none', marker=markers[i], mfc='none')
    
    ax[1].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
    ax[1].set_xscale('log')
    ax[1].legend()

ax[0].set_ylabel(r'$T_{e,probe}$ / $T_{e,TS}$')
ax[0].set_xlabel(r'$n_{e,TS}$ (m$^{-3}$)')
ax[1].set_ylabel(r'$n_{e,probe}$ / $n_{e,TS}$')
ax[1].set_xlabel(r'$n_{e,TS}$ (m$^{-3}$)')

KeyError: 'temp'

### Plot the Temperature Overestimation as a Function of Heat Flux

Among other things...

In [26]:
fig, ax = plt.subplots(2, 2, sharex='col')

for i in range(2):
#     ts_scan = combined_ds.swap_dims({'shot_number':'adc_index'}).sel(adc_index=scan_indices[i]) #.max('ts_radial_pos')
    ts_scan = combined_ds.swap_dims({'shot_number':'tilt'}).sel(tilt=scan_tilts[i]).max('ts_radial_pos')
    s_shots_scan = s_shots.sel(shot_number=ts_scan['shot_number']).max('ts_radial_pos')
    l_shots_scan = l_shots.sel(shot_number=ts_scan['shot_number']).max('ts_radial_pos')
    
    s_q_par = (s_shots_scan['isat'].values / magnum_probes['S'].get_collection_area(np.radians(scan_tilts[i]))) * 7 * s_shots_scan['temp'].values
    l_q_par = (l_shots_scan['isat'].values / magnum_probes['L'].get_collection_area(np.radians(scan_tilts[i]))) * 7 * l_shots_scan['temp'].values
    ts_q_par = ((lp.sound_speed(s_shots_scan['ts_temp'].astype(np.float64).values) 
                 * s_shots_scan['ts_dens'].values * 1.6e-19) * 7 * s_shots_scan['ts_temp'].values)
    
    # TS Density vs Probe Temperature
    ax[0][0].errorbar(s_shots_scan['ts_density'], s_shots_scan['temp'], 
                      yerr=s_shots_scan['d_temp'], label=slabel,
                      color=colours['S'], ecolor=colours['S'], 
                      linestyle='none', marker=markers[i], mfc='none')
    ax[0][0].errorbar(l_shots_scan['ts_density'], l_shots_scan['temp'], 
                      yerr=l_shots_scan['d_temp'], label=slabel,
                      color=colours['L'], ecolor=colours['L'], 
                      linestyle='none', marker=markers[i], mfc='none')
    ax[0][0].errorbar(ts_scan['ts_density'], s_shots_scan['ts_temperature'], 
                      yerr=ts_scan['ts_d_temperature'].values, 
                      color=colours['TS'], ecolor=colours['TS'], 
                      label=tslabel, linestyle='none', 
                      marker=markers[i], mfc='none')
    ax[0][0].set_xscale('log')
    ax[0][0].legend()

    # TS Heat flux vs Probe Temperature
    ax[0][1].errorbar(ts_q_par, s_shots_scan['temp'], 
                      yerr=s_shots_scan['d_temp'], label=slabel,
                      color=colours['S'], ecolor=colours['S'], 
                      linestyle='none', marker=markers[i], mfc='none')
    ax[0][1].errorbar(ts_q_par, l_shots_scan['temp'], 
                      yerr=l_shots_scan['d_temp'], label=llabel,
                      color=colours['L'], ecolor=colours['L'], 
                      linestyle='none', marker=markers[i], mfc='none')
    ax[0][1].errorbar(ts_q_par, ts_scan['ts_temperature'], 
                      yerr=ts_scan['ts_d_temperature'].values,
                      color=colours['TS'], ecolor=colours['TS'], 
                      label=tslabel, linestyle='none', 
                      marker=markers[i], mfc='none')
    ax[0][1].set_yscale('log')
    ax[0][1].set_xscale('log')
    ax[0][1].legend()

    # TS Density vs Probe Heat Flux
    ax[1][0].plot(s_shots_scan['ts_density'], s_q_par,
                  label=slabel, color=colours['S'], 
                  linestyle='none', marker=markers[i], mfc='none')
    ax[1][0].plot(s_shots_scan['ts_density'], l_q_par, label=llabel, 
                  color=colours['L'], linestyle='none', marker=markers[i], mfc='none')
    ax[1][0].plot(s_shots_scan['ts_density'], ts_q_par, label=tslabel, 
                  color=colours['TS'], linestyle='none', marker=markers[i], mfc='none')
#     ax[1][0].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
    ax[1][0].set_xscale('log')
    ax[1][0].legend()

    # TS Density vs Probe Overestimation
    ax[1][1].plot(ts_q_par, s_shots_scan['temp'].values / ts_scan['ts_temperature'].values, 
                  label=slabel, color=colours['S'], 
                  linestyle='none', marker=markers[i], mfc='none')
    ax[1][1].plot(ts_q_par, l_shots_scan['temp'].values / ts_scan['ts_temperature'].values, 
                  label=llabel, color=colours['L'], linestyle='none', marker=markers[i], mfc='none')
    ax[1][1].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
    ax[1][1].set_xscale('log')
    ax[1][1].legend()
    
fig.suptitle('Plots of temperature and density measurements during the density scan at 0.8T in hydrogen')
ax[0][0].set_title('By Density')
ax[0][1].set_title('By Heat Flux')

ax[0][0].set_ylabel(r'$T_{e,probe}$ [eV]')
ax[0][1].set_ylabel(r'$T_{e,probe}$ [eV]')
ax[1][0].set_ylabel(r'$Q_{\parallel,TS}$ [Wm$^{-2}$]')
ax[1][1].set_ylabel(r'$T_{e,probe}$ / $T_{e,TS}$')
# ax[1][1].set_ylabel(r'$n_{e,probe}$ / $n_{e,TS}$')

ax[1][0].set_xlabel(r'$n_{e,TS}$ (m$^{-3}$)')
ax[1][1].set_xlabel(r'$Q_{\parallel,TS}$ [Wm$^{-2}$]')


Text(0.5, 0, '$Q_{\\parallel,TS}$ [Wm$^{-2}$]')

In [24]:
erroneous_shots = [
    139,
    292,
    293,
    294,
    295,
    296,
    297,
    298,
    299,
    300,
    301,
    302,
    303,
    304,
    315,
    316,
    317
]

In [48]:
# shots_to_plot = combined_ds.shot_number.values
# shots_to_plot = erroneous_shots
shots_to_plot = [291, 294]

for shot_number in shots_to_plot:
    fig, ax = plt.subplots(2, sharex=True)
    ds = combined_ds.sel(shot_number=shot_number)
    
    for i, probe in enumerate(['L', 'S']):
        ds.sel(probe=probe).set_coords('voltage')['current'].plot.line(x='voltage', ax=ax[i])

In [52]:
fig, ax = plt.subplots()
mgut.plot_densscan_paramspace(combined_ds.where(combined_ds.tilt == 10.0, drop=True), ax=ax)
mgut.plot_densscan_paramspace(combined_ds.where(combined_ds.tilt == 8.0, drop=True), ax=ax)
mgut.plot_densscan_paramspace(combined_ds.sel(shot_number=erroneous_shots), ax=ax)